# Welcome to HTMX cookbook

> Please note, this very much WIP. If you have certain recipes that you want to share or want to see, head to [Github](https://github.com/callmephilip/htmx-cookbook)

This cookbook is powered by [FastHTML](https://fastht.ml/). UI is based on [block.css](https://github.com/thesephist/blocks.css)

## Getting started

```
python -m venv .env
source .env/bin/activate
pip install notebook
jupyter notebook
```

The easiest way to run this is to smash `Run -> Run All Cells`. To iterate on a specific recipe, update code in the corresponding cell and rerun it

![Screenshot](./cookbook.gif)

In [ ]:
! pip install git+https://github.com/callmephilip/fasthtml.git@tweak-jupyter-integration password_strength

In [2]:
# Setup - you can largely ingnore this unless you wan to tweak how the cookbook works

import traceback
from typing import Literal
from fasthtml.common import *
from fasthtml.jupyter import *
from ui import *

# scripts
htmx,fasthtml_js = "/htmx.js", "https://cdn.jsdelivr.net/gh/answerdotai/fasthtml-js@1.0.4/fasthtml.js"
# styles + fonts
styles,blk_css,font_css = "/styles.css","https://unpkg.com/blocks.css/dist/blocks.min.css","https://fonts.googleapis.com/css2?family=IBM+Plex+Mono:wght@500;700&amp;display=swap"
hdrs = [Script(src=s) for s in [htmx, fasthtml_js]] + [Link(href=style, rel="stylesheet") for style in [blk_css,styles,font_css]]

def server_error(request: Request, exc: HTTPException): return HTMLResponse(content=''.join(traceback.format_exception(exc)), status_code=500)

app = FastJupy(default_hdrs=False, hdrs=hdrs, exception_handlers={ 500: server_error, Exception: server_error })
rt, server = app.route, JupyUvi(app, port=8000)

@rt("/htmx.js")
def get():
    with open("htmx.js") as f:
        return f.read()
@rt("/styles.css")
def get():
    with open("styles.css") as f:
        return Response(f.read(), media_type="text/css")

In [3]:
@rt(path="/")
def get():
    return Playground(
        H1("HTMX cookbook"),
        P("This is very much WIP. Please leave your comments/requests on Github"),
        Div("Let's get started 👇", cls="wrapper animate-bounce block"),
        path="/"
    )

HTMX(path="/")

# Basic action

Grab some stuff from a URL on click

In [4]:
@rt("/data")
def get(): return Span("😱 i am trapped inside the button now!")

@rt(path="/get-on-click")
def get(): return Playground(ActionButton("Get some data", hx_get="/data"), path="/get-on-click")

HTMX(path="/get-on-click")

# Custom trigger attribute 

In [5]:
from time import sleep

@dataclass
class Html2FT: html: str

@rt("/html2ft/convert")
def post(data: Html2FT): 
    sleep(5)
    return html2ft(data.html) 

on_paste = """
document.querySelector("#txt-html").addEventListener("paste", function (event) { setTimeout(() => {event.target.blur();}, 500) });
"""

@rt("/html2ft")
def get():
    return Playground(
        Div(
            Div(
                P("Convert HTML to FT"),
                MultilineTextbox(name="html", placeholder="paste html here", cls="w-100 h-80", id="txt-html", hx_post="/html2ft/convert", hx_trigger="blur", hx_target="#result", hx_indicator="#processing-indicator"),
                Strong("Processing...", id="processing-indicator", cls="htmx-indicator"),
                cls="flex-grow-1 h-300px"
            ),
            Div(
                Code(id="result", cls="white-space-pre-wrap text-align-left"),
                cls="flex-grow-2 h-300px text-align-left", style="padding-left: 5rem;"
            ),
            cls="flex flex-row"
        ),
        Script(code=on_paste),
        path="/html2ft"
    )

HTMX(path="/html2ft")

# You can also post some data over

In [6]:
from password_strength import PasswordStats

@dataclass
class LoginData: email: str; pw:str 

@rt("/login")
def post(ld: LoginData): return Span(f"You are logged in!")

@rt("/login/validation")
def post(ld: LoginData): 
    strength = int(PasswordStats(ld.pw).strength() * 10) if ld.pw else 0
    if strength > 7:
        c = "green"
    elif strength > 5:
        c = "orange"
    else: c = "red"
    
    return Div("✅" if "@" in ld.email else "🔴", id="email-validator", hx_swap_oob="true"), strength > 0 and Div(
        Div(*[Div("+", cls=f"text-{c}") if i + 1 <= strength else Div("_", cls="tex-gray") for i in range(9)], cls="flex flex-row"),
        id="password-validator",
        hx_swap_oob="true"
    )

@rt("/login")
def get():
    return Playground(
        Form(
            Div(
                Textbox(name="email", placeholder="Email", hx_post="/login/validation", hx_trigger="keyup"), 
                Div(id="email-validator"),
                cls="flex flex-row"
            ),
            Div(
                Password(name="pw", placeholder="Password", hx_post="/login/validation", hx_trigger="keyup"),
                Div(id="password-validator"),
                cls="flex flex-row"
            ),
            ActionButton("Go!"),
            hx_post="/login"
        ), 
        path="/login")
 
HTMX(path="/login")